In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from scipy.constants import R
from openap import prop
from pprint import pprint

In [3]:
df_challenge = pd.read_csv('../data/challenge_set.csv')
df_trajectory = pd.read_parquet('../data/2022-01-01.parquet')

In [4]:
df_challenge = df_challenge.loc[df_challenge['date'] < '2022-01-05']

In [5]:
df_challenge

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,2022-01-01T15:04:56Z,A320,M,a73f82288988b79be490c6322f4c32ed,61,18,321,54748.000000
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01T09:55:00Z,2022-01-01T19:37:56Z,B772,H,5543e4dc327359ffaf5b9c0e6faaf0e1,570,13,4193,185441.000000
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01T09:39:00Z,2022-01-01T19:08:13Z,A333,H,8be5c854fd664bcb97fb543339f74770,554,15,3770,230396.000000
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01T11:04:00Z,2022-01-01T19:32:13Z,B788,H,5543e4dc327359ffaf5b9c0e6faaf0e1,497,11,3607,157615.000000
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01T12:36:00Z,2022-01-01T13:44:32Z,A21N,M,a73f82288988b79be490c6322f4c32ed,55,14,305,70318.447226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,248811363,2022-01-04,e2797e24927406797a6289ef4c87dd44,LTFM,iGA Istanbul,TR,ESSA,Stockholm Arlanda,SE,2022-01-04T06:11:00Z,2022-01-04T10:15:14Z,A21N,M,6351ec1b849adacc0cbb3b1313d8d39b,219,25,1278,78707.000000
3042,248811365,2022-01-04,d4a9a22820240e49df5ece4c1ec23de6,ESSA,Stockholm Arlanda,SE,LEMG,Malaga,ES,2022-01-04T06:06:27Z,2022-01-04T10:58:22Z,A20N,M,8be5c854fd664bcb97fb543339f74770,277,15,1723,71813.000000
3043,248826805,2022-01-04,0fa8548771a1a4cdd6de2f41fc061d7d,LEAL,Alicante,ES,EGCC,Manchester,GB,2022-01-04T18:59:00Z,2022-01-04T21:32:33Z,B738,M,3922524069809ac4326134429751e26f,144,10,1003,59816.000000
3044,248824741,2022-01-04,6e2cff31ee9ea758af1a9f6c49c9977a,LSZH,Zurich,CH,EGLL,London Heathrow,GB,2022-01-04T06:49:00Z,2022-01-04T08:27:09Z,A21N,M,2d5def0a5a844b343ba1b7cc9cb28fa9,89,9,441,70839.000000


## Data prep

In [6]:
df_challenge = df_challenge.ffill()

## Feature Encoding:

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [8]:
le.fit(df_challenge['adep'])
df_challenge['adep_category'] = le.transform(df_challenge['adep']) + 1

In [9]:
le.fit(df_challenge['ades'])
df_challenge['ades_category'] = le.transform(df_challenge['ades']) + 1

In [10]:
def encode_datetime(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [11]:
df_challenge['actual_offblock_time'] = pd.to_datetime(df_challenge['actual_offblock_time'], utc=True)
df_challenge['actual_offblock_time_month'] = df_challenge["actual_offblock_time"].dt.month
df_challenge['actual_offblock_time_day'] = df_challenge["actual_offblock_time"].dt.day
df_challenge['actual_offblock_time_hour'] = df_challenge["actual_offblock_time"].dt.hour
df_challenge['actual_offblock_time_minue'] = df_challenge["actual_offblock_time"].dt.minute
encode_datetime(df_challenge, 'actual_offblock_time_month', 12)
encode_datetime(df_challenge, 'actual_offblock_time_day', 31)
encode_datetime(df_challenge, 'actual_offblock_time_hour', 24)
encode_datetime(df_challenge, 'actual_offblock_time_minue', 60)

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,actual_offblock_time_hour,actual_offblock_time_minue,actual_offblock_time_month_sin,actual_offblock_time_month_cos,actual_offblock_time_day_sin,actual_offblock_time_day_cos,actual_offblock_time_hour_sin,actual_offblock_time_hour_cos,actual_offblock_time_minue_sin,actual_offblock_time_minue_cos
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,13,46,0.5,0.866025,0.201299,0.979530,-2.588190e-01,-9.659258e-01,-0.994522,0.104528
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01 09:55:00+00:00,...,9,55,0.5,0.866025,0.201299,0.979530,7.071068e-01,-7.071068e-01,-0.500000,0.866025
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01 09:39:00+00:00,...,9,39,0.5,0.866025,0.201299,0.979530,7.071068e-01,-7.071068e-01,-0.809017,-0.587785
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01 11:04:00+00:00,...,11,4,0.5,0.866025,0.201299,0.979530,2.588190e-01,-9.659258e-01,0.406737,0.913545
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01 12:36:00+00:00,...,12,36,0.5,0.866025,0.201299,0.979530,1.224647e-16,-1.000000e+00,-0.587785,-0.809017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,248811363,2022-01-04,e2797e24927406797a6289ef4c87dd44,LTFM,iGA Istanbul,TR,ESSA,Stockholm Arlanda,SE,2022-01-04 06:11:00+00:00,...,6,11,0.5,0.866025,0.724793,0.688967,1.000000e+00,6.123234e-17,0.913545,0.406737
3042,248811365,2022-01-04,d4a9a22820240e49df5ece4c1ec23de6,ESSA,Stockholm Arlanda,SE,LEMG,Malaga,ES,2022-01-04 06:06:27+00:00,...,6,6,0.5,0.866025,0.724793,0.688967,1.000000e+00,6.123234e-17,0.587785,0.809017
3043,248826805,2022-01-04,0fa8548771a1a4cdd6de2f41fc061d7d,LEAL,Alicante,ES,EGCC,Manchester,GB,2022-01-04 18:59:00+00:00,...,18,59,0.5,0.866025,0.724793,0.688967,-1.000000e+00,-1.836970e-16,-0.104528,0.994522
3044,248824741,2022-01-04,6e2cff31ee9ea758af1a9f6c49c9977a,LSZH,Zurich,CH,EGLL,London Heathrow,GB,2022-01-04 06:49:00+00:00,...,6,49,0.5,0.866025,0.724793,0.688967,1.000000e+00,6.123234e-17,-0.913545,0.406737


In [12]:
df_challenge['arrival_time'] = pd.to_datetime(df_challenge['arrival_time'], utc=True)
df_challenge['arrival_time_month'] = df_challenge["arrival_time"].dt.month
df_challenge['arrival_time_day'] = df_challenge["arrival_time"].dt.day
df_challenge['arrival_time_hour'] = df_challenge["arrival_time"].dt.hour
df_challenge['arrival_time_minue'] = df_challenge["arrival_time"].dt.minute
encode_datetime(df_challenge, 'arrival_time_month', 12)
encode_datetime(df_challenge, 'arrival_time_day', 31)
encode_datetime(df_challenge, 'arrival_time_hour', 12)
encode_datetime(df_challenge, 'arrival_time_minue', 60)

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,arrival_time_hour,arrival_time_minue,arrival_time_month_sin,arrival_time_month_cos,arrival_time_day_sin,arrival_time_day_cos,arrival_time_hour_sin,arrival_time_hour_cos,arrival_time_minue_sin,arrival_time_minue_cos
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,15,4,0.5,0.866025,0.201299,0.979530,1.000000,1.194340e-15,0.406737,9.135455e-01
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01 09:55:00+00:00,...,19,37,0.5,0.866025,0.201299,0.979530,-0.500000,-8.660254e-01,-0.669131,-7.431448e-01
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01 09:39:00+00:00,...,19,8,0.5,0.866025,0.201299,0.979530,-0.500000,-8.660254e-01,0.743145,6.691306e-01
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01 11:04:00+00:00,...,19,32,0.5,0.866025,0.201299,0.979530,-0.500000,-8.660254e-01,-0.207912,-9.781476e-01
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01 12:36:00+00:00,...,13,44,0.5,0.866025,0.201299,0.979530,0.500000,8.660254e-01,-0.994522,-1.045285e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,248811363,2022-01-04,e2797e24927406797a6289ef4c87dd44,LTFM,iGA Istanbul,TR,ESSA,Stockholm Arlanda,SE,2022-01-04 06:11:00+00:00,...,10,15,0.5,0.866025,0.724793,0.688967,-0.866025,5.000000e-01,1.000000,2.832769e-16
3042,248811365,2022-01-04,d4a9a22820240e49df5ece4c1ec23de6,ESSA,Stockholm Arlanda,SE,LEMG,Malaga,ES,2022-01-04 06:06:27+00:00,...,10,58,0.5,0.866025,0.724793,0.688967,-0.866025,5.000000e-01,-0.207912,9.781476e-01
3043,248826805,2022-01-04,0fa8548771a1a4cdd6de2f41fc061d7d,LEAL,Alicante,ES,EGCC,Manchester,GB,2022-01-04 18:59:00+00:00,...,21,32,0.5,0.866025,0.724793,0.688967,-1.000000,-4.286264e-16,-0.207912,-9.781476e-01
3044,248824741,2022-01-04,6e2cff31ee9ea758af1a9f6c49c9977a,LSZH,Zurich,CH,EGLL,London Heathrow,GB,2022-01-04 06:49:00+00:00,...,8,27,0.5,0.866025,0.724793,0.688967,-0.866025,-5.000000e-01,0.309017,-9.510565e-01


In [13]:
le.fit(df_challenge['aircraft_type'])
df_challenge['aircraft_type_category'] = le.transform(df_challenge['aircraft_type']) + 1

In [14]:
le.fit(df_challenge['wtc'])
df_challenge['wtc_category'] = le.transform(df_challenge['wtc']) + 1

In [15]:
le.fit(df_challenge['airline'])
df_challenge['airline_category'] = le.transform(df_challenge['airline']) + 1

In [16]:
df_challenge

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,arrival_time_month_cos,arrival_time_day_sin,arrival_time_day_cos,arrival_time_hour_sin,arrival_time_hour_cos,arrival_time_minue_sin,arrival_time_minue_cos,aircraft_type_category,wtc_category,airline_category
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01 13:46:00+00:00,...,0.866025,0.201299,0.979530,1.000000,1.194340e-15,0.406737,9.135455e-01,4,2,12
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01 09:55:00+00:00,...,0.866025,0.201299,0.979530,-0.500000,-8.660254e-01,-0.669131,-7.431448e-01,17,1,7
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01 09:39:00+00:00,...,0.866025,0.201299,0.979530,-0.500000,-8.660254e-01,0.743145,6.691306e-01,7,1,10
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01 11:04:00+00:00,...,0.866025,0.201299,0.979530,-0.500000,-8.660254e-01,-0.207912,-9.781476e-01,19,1,7
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01 12:36:00+00:00,...,0.866025,0.201299,0.979530,0.500000,8.660254e-01,-0.994522,-1.045285e-01,2,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,248811363,2022-01-04,e2797e24927406797a6289ef4c87dd44,LTFM,iGA Istanbul,TR,ESSA,Stockholm Arlanda,SE,2022-01-04 06:11:00+00:00,...,0.866025,0.724793,0.688967,-0.866025,5.000000e-01,1.000000,2.832769e-16,2,2,9
3042,248811365,2022-01-04,d4a9a22820240e49df5ece4c1ec23de6,ESSA,Stockholm Arlanda,SE,LEMG,Malaga,ES,2022-01-04 06:06:27+00:00,...,0.866025,0.724793,0.688967,-0.866025,5.000000e-01,-0.207912,9.781476e-01,1,2,10
3043,248826805,2022-01-04,0fa8548771a1a4cdd6de2f41fc061d7d,LEAL,Alicante,ES,EGCC,Manchester,GB,2022-01-04 18:59:00+00:00,...,0.866025,0.724793,0.688967,-1.000000,-4.286264e-16,-0.207912,-9.781476e-01,14,2,3
3044,248824741,2022-01-04,6e2cff31ee9ea758af1a9f6c49c9977a,LSZH,Zurich,CH,EGLL,London Heathrow,GB,2022-01-04 06:49:00+00:00,...,0.866025,0.724793,0.688967,-0.866025,-5.000000e-01,0.309017,-9.510565e-01,2,2,2


### Barometric Formula:

The air pressure $P$ at altitude $h$ is given by:

$$
P(h) = P_0 \cdot \exp\left( \frac{-Mgh}{RT} \right)
$$

Where:

- $P(h)$ is the pressure at altitude $h$,
- $P_0$ is the pressure at sea level (standard pressure: 101325 Pa),
- $M$ is the molar mass of Earth's air (approximately $0.029 \, \text{kg/mol}$),
- $g$ is the acceleration due to gravity $(9.81 \, \text{m/s}^2)$,
- $h$ is the altitude (in meters),
- $R$ is the universal gas constant $(8.314 \, \text{J/(mol·K)})$,
- $T$ is the temperature at altitude $h$ (in Kelvin).

### Air Density Formula:

The air density $rho$ is given by the ideal gas law:

$$
\rho = \frac{P}{R_s T}
$$

Where:
- $P$ is the pressure at altitude $h$,
- $R_s$ is the specific gas constant for dry air (approximately $287.05 \, \text{J/(kg·K)}$),
- $T$ is the temperature in Kelvin.

### Combined Formula for Air Density:

$$
\rho(h) = \frac{P_0 \cdot \exp\left( \frac{-Mgh}{RT} \right)}{R_s T}
$$

In [17]:
# Constants
p0 = 101325  # Pressure at sea level (Pa)
T0 = 288.15  # Standard temperature at sea level (K)
g = 9.80665  # Acceleration due to gravity (m/s²)
L = 0.0065  # Temperature lapse rate (K/m)
R = 8.3144598  # Universal gas constant (J/(mol·K))
M = 0.0289644  # Molar mass of Earth's air (kg/mol)
R_air = 287.05  # Specific gas constant for dry air (J/(kg·K))
C_L = 0.5  # Approximate lift coefficient (this can vary, but 0.5 is a reasonable starting point for climb)

In [18]:
# Function to calculate air density with humidity
def air_density_with_humidity(altitude, temperature, specific_humidity):
    temperature_kelvin = temperature + 273.15
    
    # Calculate pressure at altitude using barometric formula
    pressure = p0 * (1 - (L * altitude) / T0) ** (g * M / (R * L))
    
    # Calculate partial pressure of water vapor (e) using specific humidity
    e = (specific_humidity * pressure) / (0.622 + specific_humidity)
    
    # Calculate virtual temperature (T_v)
    virtual_temperature = temperature_kelvin * (1 + 0.61 * specific_humidity)
    
    # Calculate air density using the virtual temperature
    density = pressure / (R_air * virtual_temperature)
    
    return density    

## Wind speed
v = $\sqrt[2]{u\_wind^2+v\_wind^2}$

In [19]:
def wind_speed(u_wind, v_wind):
    return np.sqrt(u_wind **2 + v_wind ** 2)

## True Air speed
v = $\sqrt[2]{groundspeed^2+wind\_speed^2}$

In [20]:
# Function to calculate true airspeed with altitude-based wind components
def true_airspeed(groundspeed, u_wind, v_wind):
    return np.sqrt(groundspeed**2 + wind_Speed(u_wind, v_wind))

## estimate lift force

In [21]:
# df_trajectory['wind_speed'] = df_trajectory.apply(lambda row: wind_speed(row['u_component_of_wind'], row['v_component_of_wind']), axis=1)
df_trajectory['wind_speed'] = np.sqrt(df_trajectory['u_component_of_wind'] **2 + df_trajectory['v_component_of_wind'] ** 2)

0           15.731198
1           15.729412
2           15.730149
3           15.730886
4           15.731623
              ...    
11862458    58.630812
11862459    58.630692
11862460    58.630572
11862461    58.630452
11862462    58.630333
Name: wind_speed, Length: 11862463, dtype: float64

In [22]:
df_trajectory['true_airspeed'] = np.sqrt(df_trajectory['groundspeed'] **2 + df_trajectory['wind_speed'] ** 2)

0           429.288330
1           429.288265
2           429.288292
3           429.288319
4           429.288346
               ...    
11862458    546.156179
11862459    546.156166
11862460    546.156153
11862461    546.156141
11862462    546.156128
Name: true_airspeed, Length: 11862463, dtype: float64

In [23]:
df_trajectory['pressure'] = p0 * (1 - (L * df_trajectory['altitude']) / T0) ** (g * M / (R * L))

0           3.658208
1           3.658208
2           3.658208
3           3.658208
4           3.658208
              ...   
11862458    0.130147
11862459    0.130147
11862460    0.130147
11862461    0.130147
11862462    0.130147
Name: pressure, Length: 11862463, dtype: float64

In [24]:
# Calculate partial pressure of water vapor (e) using specific humidity
df_trajectory['pressure_e'] = (df_trajectory['specific_humidity'] * df_trajectory['pressure']) / (0.622 + df_trajectory['specific_humidity'])

0           0.000045
1           0.000045
2           0.000045
3           0.000045
4           0.000045
              ...   
11862458    0.000003
11862459    0.000003
11862460    0.000003
11862461    0.000003
11862462    0.000003
Name: pressure_e, Length: 11862463, dtype: float64

In [25]:
# Calculate virtual temperature (T_v)
df_trajectory['virtual_temperature'] = (df_trajectory['temperature']+ 273.15) * (1 + 0.61 * df_trajectory['specific_humidity'])

0           494.971907
1           494.947078
2           494.946996
3           494.946915
4           494.946833
               ...    
11862458    483.362372
11862459    483.362361
11862460    483.362350
11862461    483.362339
11862462    483.362328
Name: virtual_temperature, Length: 11862463, dtype: float64

In [26]:
df_trajectory['air_density'] = df_trajectory['pressure'] / (R_air * df_trajectory['virtual_temperature'])

0           2.574722e-05
1           2.574851e-05
2           2.574851e-05
3           2.574852e-05
4           2.574852e-05
                ...     
11862458    9.380035e-07
11862459    9.380036e-07
11862460    9.380036e-07
11862461    9.380036e-07
11862462    9.380036e-07
Name: air_density, Length: 11862463, dtype: float64

In [27]:
# Assume K is some constant value based on typical aircraft performance
# (Note: This needs to be estimated or assumed. We'll use a placeholder value for now)
K = 0.5  # A reasonable placeholder value for K based on aircraft type

# Estimate lift force using L = K * rho * v^2
df_trajectory['lift_force'] = K * df_trajectory['air_density'] * df_trajectory['true_airspeed']**2

0           2.372458
1           2.372576
2           2.372577
3           2.372577
4           2.372578
              ...   
11862458    0.139897
11862459    0.139897
11862460    0.139897
11862461    0.139897
11862462    0.139897
Name: lift_force, Length: 11862463, dtype: float64

In [53]:
# Categorical features to one-hot encode
categorical_features = [
    'flight_duration',
    'flown_distance', 
    'adep_category',
    'ades_category',
    'actual_offblock_time_month',
    'actual_offblock_time_day',
    'actual_offblock_time_hour',
    'actual_offblock_time_minue',
    'actual_offblock_time_month_sin',
    'actual_offblock_time_day_sin',
    'actual_offblock_time_day_cos',
    'actual_offblock_time_hour_sin',
    'actual_offblock_time_hour_cos',
    'actual_offblock_time_minue_sin',
    'actual_offblock_time_minue_cos',
    'arrival_time_month',
    'arrival_time_day',
    'arrival_time_hour',
    'arrival_time_minue',
    'arrival_time_month_cos',
    'arrival_time_day_cos',
    'arrival_time_hour_sin',
    'arrival_time_hour_cos',
    'arrival_time_minue_sin',
    'arrival_time_minue_cos',
    'aircraft_type_category',
    'wtc_category',
    'airline_category'
]

trajectory_features = ['altitude', 'groundspeed', 'vertical_rate','true_airspeed', 'air_density','lift_force', 'temperature', 'specific_humidity', 'wind_speed']

# Create an empty dataframe to store aggregated trajectory data
aggregated_trajectory = pd.DataFrame()

# Group trajectory data by flight_id to compute per-flight aggregates
for flight_id, group in df_trajectory.groupby('flight_id'):
    aggregated_data = {}
    aggregated_data['flight_id'] = flight_id
    
    for feature in trajectory_features:
        aggregated_data[f'{feature}_mean'] = group[feature].mean()
        aggregated_data[f'{feature}_std'] = group[feature].std()
        aggregated_data[f'{feature}_min'] = group[feature].min()
        aggregated_data[f'{feature}_max'] = group[feature].max()
    # Append aggregated data to the aggregated_trajectory dataframe
    aggregated_trajectory =  pd.concat([aggregated_trajectory, pd.DataFrame.from_dict(aggregated_data, orient='index').T])

# ----- Merge Static and Aggregated Trajectory Data -----
# Merge the static flight data and the aggregated trajectory data on flight_id
# df_combined = aggregated_trajectory.join(df_challenge.set_index('flight_id'), on='flight_id')
df_combined = df_challenge.combine_first(aggregated_trajectory)
# # ----- Prepare Features and Target for Model -----
# # Define the features and target variable (TOW)
features = [col for col in df_combined.columns if col not in ['tow', 'flight_id']]  # 'tow' is the target
X = df_combined[features]
y = df_combined['tow']

In [54]:
df_combined

,actual_offblock_time,actual_offblock_time_day,actual_offblock_time_day_cos,actual_offblock_time_day_sin,actual_offblock_time_hour,actual_offblock_time_hour_cos,actual_offblock_time_hour_sin,actual_offblock_time_minue,actual_offblock_time_minue_cos,actual_offblock_time_minue_sin,...,vertical_rate_max,vertical_rate_mean,vertical_rate_min,vertical_rate_std,wind_speed_max,wind_speed_mean,wind_speed_min,wind_speed_std,wtc,wtc_category
0,2022-01-01 13:46:00+00:00,1,0.979530,0.201299,13,-9.659258e-01,-2.588190e-01,46,0.104528,-0.994522,...,5440.0,83.381810,-3840.0,804.041495,45.881548,13.974705,7.208165,4.939622,M,2
0,2022-01-01 13:46:00+00:00,1,0.979530,0.201299,13,-9.659258e-01,-2.588190e-01,46,0.104528,-0.994522,...,64.0,-826.598691,-4736.0,823.578926,30.496333,17.619324,3.875787,9.818136,M,2
0,2022-01-01 13:46:00+00:00,1,0.979530,0.201299,13,-9.659258e-01,-2.588190e-01,46,0.104528,-0.994522,...,2688.0,553.936652,-64.0,775.667346,26.621479,16.602471,4.387229,5.980834,M,2
0,2022-01-01 13:46:00+00:00,1,0.979530,0.201299,13,-9.659258e-01,-2.588190e-01,46,0.104528,-0.994522,...,2944.0,244.192761,-4608.0,971.069513,25.603216,15.696475,1.968458,6.206049,M,2
0,2022-01-01 13:46:00+00:00,1,0.979530,0.201299,13,-9.659258e-01,-2.588190e-01,46,0.104528,-0.994522,...,2752.0,1666.945006,896.0,379.158169,29.640263,19.059297,1.598454,8.220425,M,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,2022-01-04 06:11:00+00:00,4,0.688967,0.724793,6,6.123234e-17,1.000000e+00,11,0.406737,0.913545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,2
3042,2022-01-04 06:06:27+00:00,4,0.688967,0.724793,6,6.123234e-17,1.000000e+00,6,0.809017,0.587785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,2
3043,2022-01-04 18:59:00+00:00,4,0.688967,0.724793,18,-1.836970e-16,-1.000000e+00,59,0.994522,-0.104528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,2
3044,2022-01-04 06:49:00+00:00,4,0.688967,0.724793,6,6.123234e-17,1.000000e+00,49,0.406737,-0.913545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,2


# Feature Encoding:

## Model

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
from lightgbm import LGBMRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import root_mean_squared_error, r2_score


model = LGBMRegressor(learning_rate=0.5,force_row_wise=True)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print("model score", model.score(x_train, y_train))
# 0.9863556751160256

print("rmse:", root_mean_squared_error(y_test, y_pred))
print("r2 score", r2_score(y_test, y_pred))

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: date: object, callsign: object, adep: object, name_adep: object, country_code_adep: object, ades: object, name_ades: object, country_code_ades: object, actual_offblock_time: datetime64[ns, UTC], arrival_time: datetime64[ns, UTC], aircraft_type: object, wtc: object, airline: object